In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from functools import partial
import optuna
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

use_stack = False
new_study = False
do_lda = False
fix_data_skew = False

In [2]:
train_df = pd.read_csv('input/train' + ('_stack' if use_stack else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_stack' if use_stack else '') + '.csv')

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
train_df.head()

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(X, y, trial):
    score = 0.0

    # based on kernel params
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": "gbdt",
        "max_depth" : -1,
        "num_leaves" : trial.suggest_int('num_leaves', 14, 16),
        "learning_rate" : 0.01,
        "bagging_freq": trial.suggest_int('bagging_freq', 3, 5),
        "bagging_fraction" : trial.suggest_uniform('bagging_fraction', 0.45, 0.52),
        "feature_fraction" : trial.suggest_uniform('feature_fraction', 0.038, 0.043), 
        "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 73, 78),
        "min_sum_heassian_in_leaf": trial.suggest_int('min_sum_heassian_in_leaf', 11, 13),
        "tree_learner": "serial", # "serial",
        "num_threads": 6,
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
            
        model = lgb.train(params,train_data,num_boost_round=20000,
                        valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds=100)
        score += model.best_score['valid_1']['auc'] / n_splits
    print(score)
    return 1.0 - score

In [5]:
def main():
    f = partial(objective, X, y)
    if new_study:
        study = optuna.create_study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    else:
        study = optuna.Study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    study.optimize(f, n_trials=300)
    print('params:', study.best_params)

if __name__ == '__main__':
    main()

Fold 0 started at Thu Apr  4 16:43:42 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.889902	valid_1's auc: 0.870257
[600]	training's auc: 0.902379	valid_1's auc: 0.881256
[900]	training's auc: 0.908426	valid_1's auc: 0.886812
[1200]	training's auc: 0.912	valid_1's auc: 0.889439
[1500]	training's auc: 0.914763	valid_1's auc: 0.891229
[1800]	training's auc: 0.917298	valid_1's auc: 0.89284
[2100]	training's auc: 0.919556	valid_1's auc: 0.894166
[2400]	training's auc: 0.921957	valid_1's auc: 0.895182
[2700]	training's auc: 0.92408	valid_1's auc: 0.896054
[3000]	training's auc: 0.926059	valid_1's auc: 0.896647
[3300]	training's auc: 0.928014	valid_1's auc: 0.897084
[3600]	training's auc: 0.929728	valid_1's auc: 0.897412
[3900]	training's auc: 0.931387	valid_1's auc: 0.897732
[4200]	training's auc: 0.933077	valid_1's auc: 0.898118
[4500]	training's auc: 0.934664	valid_1's auc: 0.898359
[4800]	training's auc: 0.936336	valid_1's auc: 0.898618
[5100]

[I 2019-04-04 16:48:22,661] Finished a trial resulted in value: 0.10021666130946627. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


Fold 0 started at Thu Apr  4 16:48:24 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.88953	valid_1's auc: 0.870466
[600]	training's auc: 0.90196	valid_1's auc: 0.88227
[900]	training's auc: 0.908547	valid_1's auc: 0.887867
[1200]	training's auc: 0.912099	valid_1's auc: 0.890987
[1500]	training's auc: 0.914766	valid_1's auc: 0.892411
[1800]	training's auc: 0.917005	valid_1's auc: 0.893707
[2100]	training's auc: 0.919103	valid_1's auc: 0.89463
[2400]	training's auc: 0.920934	valid_1's auc: 0.895493
[2700]	training's auc: 0.922867	valid_1's auc: 0.896211
[3000]	training's auc: 0.924655	valid_1's auc: 0.896901
[3300]	training's auc: 0.926301	valid_1's auc: 0.897393
[3600]	training's auc: 0.927951	valid_1's auc: 0.897841
[3900]	training's auc: 0.929554	valid_1's auc: 0.898256
[4200]	training's auc: 0.931013	valid_1's auc: 0.898583
[4500]	training's auc: 0.932423	valid_1's auc: 0.898846
[4800]	training's auc: 0.933896	valid_1's auc: 0.899122
Early

[I 2019-04-04 16:52:43,755] Finished a trial resulted in value: 0.10012905037377418. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


Fold 0 started at Thu Apr  4 16:52:45 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.89128	valid_1's auc: 0.870836
[600]	training's auc: 0.903701	valid_1's auc: 0.882506
[900]	training's auc: 0.910303	valid_1's auc: 0.888026
[1200]	training's auc: 0.913883	valid_1's auc: 0.89108
[1500]	training's auc: 0.916612	valid_1's auc: 0.892779
[1800]	training's auc: 0.918818	valid_1's auc: 0.894218
[2100]	training's auc: 0.920947	valid_1's auc: 0.895071
[2400]	training's auc: 0.922819	valid_1's auc: 0.89591
[2700]	training's auc: 0.924779	valid_1's auc: 0.896579
[3000]	training's auc: 0.926626	valid_1's auc: 0.897105
[3300]	training's auc: 0.928342	valid_1's auc: 0.897704
[3600]	training's auc: 0.930036	valid_1's auc: 0.898041
[3900]	training's auc: 0.931696	valid_1's auc: 0.898462
[4200]	training's auc: 0.933253	valid_1's auc: 0.898668
[4500]	training's auc: 0.93471	valid_1's auc: 0.89887
[4800]	training's auc: 0.936211	valid_1's auc: 0.899145
Early 

...

[I 2019-04-05 08:01:48,055] Finished a trial resulted in value: 0.10016239739626331. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Fri Apr  5 08:01:49 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887754	valid_1's auc: 0.869057
[600]	training's auc: 0.900456	valid_1's auc: 0.880832
[900]	training's auc: 0.906417	valid_1's auc: 0.886144
[1200]	training's auc: 0.910047	valid_1's auc: 0.888651
[1500]	training's auc: 0.912833	valid_1's auc: 0.890576
[1800]	training's auc: 0.915304	valid_1's auc: 0.892138
[2100]	training's auc: 0.917608	valid_1's auc: 0.893434
[2400]	training's auc: 0.91997	valid_1's auc: 0.89448
[2700]	training's auc: 0.922052	valid_1's auc: 0.895436
[3000]	training's auc: 0.923991	valid_1's auc: 0.896018
[3300]	training's auc: 0.925921	valid_1's auc: 0.896584
[3600]	training's auc: 0.927626	valid_1's auc: 0.897014
[3900]	training's auc: 0.929232	valid_1's auc: 0.897557
[4200]	training's auc: 0.930834	valid_1's auc: 0.898035
[4500]	training's auc: 0.932388	valid_1's auc: 0.89841
[4800]	training's auc: 0.93397	valid_1's auc: 0.898781
[5100

[I 2019-04-05 08:06:29,992] Finished a trial resulted in value: 0.09998794082963758. Current best value is 0.09976555762908212 with parameters: {'bagging_fraction': 0.48231252804226993, 'bagging_freq': 4, 'feature_fraction': 0.04283121565948458, 'min_data_in_leaf': 76, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 14}.


Fold 0 started at Fri Apr  5 08:06:31 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887727	valid_1's auc: 0.869007
[600]	training's auc: 0.900501	valid_1's auc: 0.880492
[900]	training's auc: 0.906444	valid_1's auc: 0.886149
[1200]	training's auc: 0.910034	valid_1's auc: 0.888762
[1500]	training's auc: 0.912784	valid_1's auc: 0.890662
[1800]	training's auc: 0.915254	valid_1's auc: 0.892224
[2100]	training's auc: 0.917505	valid_1's auc: 0.893493
[2400]	training's auc: 0.919855	valid_1's auc: 0.894526
[2700]	training's auc: 0.922016	valid_1's auc: 0.895435
[3000]	training's auc: 0.923982	valid_1's auc: 0.895979
[3300]	training's auc: 0.925884	valid_1's auc: 0.896537
[3600]	training's auc: 0.927654	valid_1's auc: 0.896892
[3900]	training's auc: 0.929259	valid_1's auc: 0.897339
[4200]	training's auc: 0.930854	valid_1's auc: 0.897784
[4500]	training's auc: 0.932411	valid_1's auc: 0.898113
[4800]	training's auc: 0.933993	valid_1's auc: 0.898514
[

KeyboardInterrupt: 